In [1]:
import torch


from tqdm.notebook import tqdm

from neurovlm.text_input import search_papers_from_text, search_wiki_from_text, generate_llm_response_from_text, search_cogatlas_from_text
from neurovlm.brain_input import (
    search_papers_from_brain,
    generate_llm_response_from_brain,
    resample_networks_to_mask
)

from neurovlm.retrieval_resources import (
    _load_autoencoder,
    _load_networks,
    _load_masker )

## Ollama Setup

** 1. Install Ollama **
- ** using brew: **
```sh
brew install ollama
```

** 2. Start Ollama Service **
```
ollama serve
```

** 3. Download the ollama app **

** 4. Pull a Model(Our default model is qwen2.5:7b) **
```
ollama pull qwen2.5:7b-instruct  or ollama pull qwen2.5:3b-instruct
```


## Load data and models

In [2]:
# Load data and Specter
autoencoder = _load_autoencoder()

masker = _load_masker()

## Example query for text

The titles, abstract, cognitive concepts, cognitive tasks, and cognitive disorders most related to the query will be passed to the language model.

The function accepts a string input; no need to encode the input text before passing it in

In [3]:
query = "default mode network"

In [4]:
query

'default mode network'

In [5]:
# Get the top k similar papers
abstract, titles = search_papers_from_text(query, show_titles = True)

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [13]:
context, titles = search_wiki_from_text("dementia", show_titles = True)

Top matches:
1. Idiopathic disease
2. SLC10A2


In [14]:
definition, terms, _ = search_cogatlas_from_text(query, show_titles=True)

Top matches:
1. declarative knowledge
2. spreading activation
3. inference
4. active maintenance
5. intertemporal choice


In [15]:
definition, terms, _ = search_cogatlas_from_text(query, show_titles=True, category= "cogatlas_disorder")

Top matches:
1. mild cognitive impairment
2. impotence
3. melancholia
4. anosognosia
5. pedophilia


In [16]:
definition, terms, _ = search_cogatlas_from_text(query, show_titles=True, category= "cogatlas_task")

Top matches:
1. lateral facilitation
2. autism spectrum quotient
3. autobiographical memory task
4. local computation
5. theory of mind task


In [18]:
output = generate_llm_response_from_text(query)

Generating LLM response for query: 'default mode network'
- Using papers
- Using CogAtlas terms
LLM writing summary...
Loading model Qwen/Qwen2.5-3B-Instruct from Hugging Face...



KeyboardInterrupt



## Example query for Brain Input

The titles and abstract most related to the brain will be passed to the LM.

The functions expects an already encoded brain input

In [11]:
# Load network atlases
networks = _load_networks()

In [2]:
networks_resampled = resample_networks_to_mask(networks)

NameError: name 'resample_networks_to_mask' is not defined

In [13]:
networks_embed = {}

for k, v in tqdm(networks_resampled.items(), total=len(networks_resampled)):
    networks_embed[k] = autoencoder.encoder(torch.from_numpy(masker.transform(v)))

  0%|          | 0/152 [00:00<?, ?it/s]

In [15]:
# Look for abstract and titles related to Auditory network
abstract, titles, _ = search_papers_from_brain(networks_embed["AUD"], show_titles=True)

Top matches:
1. Neural correlates of sound externalization.
2. Audio-visual synchrony modulates the ventriloquist illusion and its neural/spatial representation in the auditory cortex.
3. Processing of spectral and amplitude envelope of animal vocalizations in the human auditory cortex.
4. Music listening engages specific cortical regions within the temporal lobes: Differences between musicians and non-musicians.
5. Processing of natural sounds in human auditory cortex: tonotopy, spectral tuning, and relation to voice sensitivity.


In [ ]:
# LLM summary for related abstracts
output = generate_llm_response_from_brain(networks_embed["AUD"])